In [ ]:
from google.colab import files
uploaded = files.upload()

Saving estudiantes_ingenieria_con_errores.xlsx to estudiantes_ingenieria_con_errores.xlsx


In [ ]:
import pandas as pd
import numpy as np
import unicodedata
import re

In [ ]:
nombre_archivo = 'estudiantes_ingenieria_con_errores.xlsx'
try:
    df = pd.read_excel(nombre_archivo)
    print("Archivo cargado exitosamente.")
except FileNotFoundError:
    print(f"Error: El archivo '{nombre_archivo}' no fue encontrado. Asegúrate de haberlo subido y que el nombre sea correcto.")
except Exception as e:
    print(f"Ocurrió un error al cargar el archivo: {e}")

Ocurrió un error al cargar el archivo: name 'pd' is not defined


In [ ]:
print("Primeras 5 filas del DataFrame original:")
print(df.head())

print("\nInformación general del DataFrame original:")
df.info()

print("\nResumen estadístico original:")
print(df.describe(include='all'))

Primeras 5 filas del DataFrame original:
  Nombre del Estudiante Edad Fecha de Nacimiento Género  \
0                   NaN  NaN                 NaN    NaN   
1             abad vega   23          04/09/2002      f   
2       abaneth lizcano   22          30/11/1988      M   
3   abderraman martinez   18          02/11/2003      M   
4     abdruval collazos   22          21/08/1969      F   

                                  Email Ciudad de Nacimiento  \
0                                   NaN                  NaN   
1                  abadvega@miemail.com            MonterÃ­a   
2      abanethlizcano@miuniversidad.com                Yumbo   
3  abderramanmartinez@miuniversidad.com             Rionegro   
4            abdruvalcollazos@mihot.com             Girardot   

  Deporte que Practica  Semestre Cursos Matriculados Créditos Aprobados  \
0                  NaN       NaN                 NaN                NaN   
1            Bádminton       1.0                   6                 

**Analisis de los resultado del DataFrame**

*   El archivo se cargó correctamente.
*   Se observa una fila inicial completamente vacía (llena de NaN).
*   Los nombres de las columnas tienen mayúsculas, espacios y tildes (ej. "Nombre del Estudiante", "Costo Matrícula").
*   La columna "Ciudad de Nacimiento" muestra caracteres extraños (ej. "MonterÃ­a"), lo que sugiere un problema de codificación o tipeo.
*   La columna "Edad" está como float64, probablemente por los NaN. Debería ser un entero.
*   Las columnas "Proyectos de Investigación" e "Interés estudiar posgrado" tienen valores como "SI" y "sÍ", necesitan estandarizarse.
*   "Costo Matrícula" y "Promedio Actual" parecen numéricos pero podrían tener formatos inconsistentes no visibles en las primeras 5 filas.


In [ ]:
print(f"Filas antes de eliminar vacías: {len(df)}")
df.dropna(how='all', inplace=True)
print(f"Filas después de eliminar vacías: {len(df)}")

Filas antes de eliminar vacías: 3804
Filas después de eliminar vacías: 3803


In [ ]:
df.replace(['*', '#N/A', 'N/D', 'nan', ''], np.nan, inplace=True)
print("Placeholders comunes como '*', 'N/D', cadenas vacías y 'nan' (texto) reemplazados por np.nan.")

Placeholders comunes como '*', 'N/D', cadenas vacías y 'nan' (texto) reemplazados por np.nan.


In [ ]:

if 'Costo Matrícula' in df.columns:
    df['Costo Matrícula'] = df['Costo Matrícula'].astype(str)
    # Eliminar puntos (que podrían ser separadores de miles)
    df['Costo Matrícula'] = df['Costo Matrícula'].str.replace('.', '', regex=False)
    print("Puntos eliminados de 'Costo Matrícula' (si existían). La conversión a numérico se hará en el Paso 4.")
else:
    print("Columna 'Costo Matrícula' no encontrada con ese nombre exacto.")

if 'Promedio Actual' in df.columns: # Uso el nombre original
    df['Promedio Actual'] = df['Promedio Actual'].astype(str)
    df['Promedio Actual'] = df['Promedio Actual'].str.replace(',', '.', regex=False)
    print("Comas reemplazadas por puntos en 'Promedio Actual' (si existían). La conversión a numérico se hará en el Paso 4.")
else:
    print("Columna 'Promedio Actual' no encontrada con ese nombre exacto.")

Puntos eliminados de 'Costo Matrícula' (si existían). La conversión a numérico se hará en el Paso 4.
Comas reemplazadas por puntos en 'Promedio Actual' (si existían). La conversión a numérico se hará en el Paso 4.


**Eliminación de Registros No Válidos y Correcciones Iniciales**

*   Se eliminaron las filas completamente vacías.
*   Se reemplazaron placeholders comunes ('*', 'N/D', 'nan' como texto, cadenas vacías) por `np.nan`.
*   Se preparó el código para eliminar puntos de 'Costo Matrícula' y estandarizar decimales en 'Promedio Actual', aunque en la muestra inicial estos problemas no eran evidentes, se sigue la guía del ejercicio.

In [ ]:
print("Nombres de columnas originales:", df.columns.tolist())

def limpiar_nombre_columna(col_name):
    col_name = str(col_name).lower() #Convertir a minúsculas
#Quitar tildes y caracteres diacríticos
    col_name = unicodedata.normalize('NFKD', col_name).encode('ascii', 'ignore').decode('utf-8')
#Reemplazar espacios con guion bajo
    col_name = col_name.replace(' ', '_')
#Eliminar cualquier caracter que no sea letra minúscula, número o guion bajo
    col_name = re.sub(r'[^a-z0-9_]', '', col_name)
    return col_name

df.columns = [limpiar_nombre_columna(col) for col in df.columns]
print("Nombres de columnas estandarizados:", df.columns.tolist())

Nombres de columnas originales: ['Nombre del Estudiante', 'Edad', 'Fecha de Nacimiento', 'Género', 'Email', 'Ciudad de Nacimiento', 'Deporte que Practica', 'Semestre', 'Cursos Matriculados', 'Créditos Aprobados', 'Proyectos de Investigación', 'Costo Matrícula', 'Promedio Actual', 'Tipo de empleo', 'Jornada Estudio', 'Interés estudiar posgrado']
Nombres de columnas estandarizados: ['nombre_del_estudiante', 'edad', 'fecha_de_nacimiento', 'genero', 'email', 'ciudad_de_nacimiento', 'deporte_que_practica', 'semestre', 'cursos_matriculados', 'creditos_aprobados', 'proyectos_de_investigacion', 'costo_matricula', 'promedio_actual', 'tipo_de_empleo', 'jornada_estudio', 'interes_estudiar_posgrado']


In [ ]:
if 'nombre_del_estudiante' in df.columns:
    df['nombre_del_estudiante'] = df['nombre_del_estudiante'].astype(str) # Asegurar que sea texto

    split_names = df['nombre_del_estudiante'].str.split(' ', n=1, expand=True)

    df['nombres'] = split_names[0] # La primera parte es el nombre
    df['apellidos'] = split_names[1] # La segunda parte (el resto) son los apellidos

    # Limpiar posibles 'nan' de texto que resultaron de la conversión a string
    df.loc[df['nombres'] == 'nan', 'nombres'] = np.nan
    df.loc[df['apellidos'] == 'nan', 'apellidos'] = np.nan
    df.loc[df['apellidos'].isnull(), 'apellidos'] = np.nan # Si no hubo segundo nombre/apellido, asegurar que sea NaN

    print("Columna 'nombre_del_estudiante' separada en 'nombres' y 'apellidos'.")
    print("Primeras filas de las nuevas columnas 'nombres' y 'apellidos':")
    print(df[['nombres', 'apellidos']].head())

else:
    print("Columna 'nombre_del_estudiante' no encontrada para separación.")

Columna 'nombre_del_estudiante' separada en 'nombres' y 'apellidos'.
Primeras filas de las nuevas columnas 'nombres' y 'apellidos':
      nombres apellidos
1        abad      vega
2     abaneth   lizcano
3  abderraman  martinez
4    abdruval  collazos
5        abel   sanchez


In [ ]:
print("\nTipos de datos después de estandarizar nombres y separar (antes de limpieza específica por columna):")
df.info()


Tipos de datos después de estandarizar nombres y separar (antes de limpieza específica por columna):
<class 'pandas.core.frame.DataFrame'>
Index: 3803 entries, 1 to 3803
Data columns (total 18 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   nombre_del_estudiante       3803 non-null   object 
 1   edad                        3800 non-null   object 
 2   fecha_de_nacimiento         3803 non-null   object 
 3   genero                      3707 non-null   object 
 4   email                       3803 non-null   object 
 5   ciudad_de_nacimiento        3803 non-null   object 
 6   deporte_que_practica        3803 non-null   object 
 7   semestre                    3803 non-null   float64
 8   cursos_matriculados         3676 non-null   object 
 9   creditos_aprobados          3777 non-null   object 
 10  proyectos_de_investigacion  3803 non-null   object 
 11  costo_matricula             3803 non-null   object

Paso 3: Estandarización de Columnas

*   Los nombres de las columnas se convirtieron a minúsculas, se quitaron tildes y caracteres especiales, y los espacios se reemplazaron por guiones bajos.
*   La columna `nombre_del_estudiante` se separó en `nombres` y `apellidos`.
*   Se revisaron los tipos de datos preliminarmente. La corrección final de tipos se hará en el siguiente paso.

In [ ]:
if 'edad' in df.columns:

    df['edad'] = pd.to_numeric(df['edad'], errors='coerce')

    df.loc[(df['edad'] <= 10) | (df['edad'] > 80), 'edad'] = np.nan
    df['edad'] = df['edad'].astype('Int64')

    print("Columna 'edad' limpiada.")
    print("Valores únicos en 'edad' (muestra):", df['edad'].unique()[:20]) # Muestra los primeros 20 valores únicos
    print("Descripción de 'edad':")
    print(df['edad'].describe())
else:
    print("Columna 'edad' no encontrada.")

Columna 'edad' limpiada.
Valores únicos en 'edad' (muestra): <IntegerArray>
[23, 22, 18, 19, 17, 20, 27, 26, 24, 30, 21, 25, 28, 31, <NA>, 29, 16]
Length: 17, dtype: Int64
Descripción de 'edad':
count       3795.0
mean     21.198419
std       3.171582
min           16.0
25%           19.0
50%           21.0
75%           23.0
max           31.0
Name: edad, dtype: Float64


In [ ]:
if 'genero' in df.columns:

    df['genero'] = df['genero'].astype(str).str.strip().str.upper()

    genero_map = {
        'F': 'F', 'FEMENINO': 'F', 'MUJER': 'F', # Valores para Femenino
        'M': 'M', 'MASCULINO': 'M', 'HOMBRE': 'M', # Valores para Masculino
        'NAN': np.nan # Si 'nan' (texto) quedó de la conversión a str
    }
    df['genero'] = df['genero'].replace(genero_map)


    df.loc[~df['genero'].isin(['M', 'F']), 'genero'] = np.nan

    print("Columna 'genero' limpiada.")
    print("Valores únicos en 'genero':", df['genero'].unique())
    print("Conteo de valores en 'genero':")
    print(df['genero'].value_counts(dropna=False)) # dropna=False para incluir conteo de NaN
else:
    print("Columna 'genero' no encontrada.")

Columna 'genero' limpiada.
Valores únicos en 'genero': ['F' 'M' nan]
Conteo de valores en 'genero':
genero
F      1603
M      1482
NaN     718
Name: count, dtype: int64


In [ ]:
if 'ciudad_de_nacimiento' in df.columns:
    df['ciudad_de_nacimiento'] = df['ciudad_de_nacimiento'].astype(str).str.strip()

    def corregir_ciudad(ciudad_str):
        if pd.isna(ciudad_str) or ciudad_str.lower() == 'nan':
            return np.nan

        ciudad_str = ciudad_str.replace('Ã\xada', 'ía').replace('Ã©', 'é')
        ciudad_str = ciudad_str.replace('Ã¡', 'á').replace('Ã³', 'ó')
        ciudad_str = ciudad_str.replace('Ãº', 'ú').replace('Ã±', 'ñ')
        ciudad_str = ciudad_str.replace('MonterÃ­a', 'Montería')
        ciudad_str = ciudad_str.replace('BogotÃ¡', 'Bogotá')

        correcciones_ciudad_map = {
            'bogota d.c.': 'Bogotá', # Estandarizar a Bogotá con tilde
            'sta marta': 'Santa Marta',
            'monteria': 'Montería', # Si después de la corrección de arriba queda sin tilde
            'yumbo': 'Yumbo'
        }
        ciudad_lower_temp = ciudad_str.lower() # Convertir a minúscula para buscar en el mapa
        if ciudad_lower_temp in correcciones_ciudad_map:
            ciudad_str = correcciones_ciudad_map[ciudad_lower_temp]

        ciudad_str = re.sub(r'[^a-zA-ZáéíóúÁÉÍÓÚñÑ\s]', '', ciudad_str).strip()

        # Capitalizar la primera letra de cada palabra (ej. "santa marta" -> "Santa Marta")
        return ciudad_str.title() if ciudad_str else np.nan

    df['ciudad_de_nacimiento'] = df['ciudad_de_nacimiento'].apply(corregir_ciudad)
    df.loc[df['ciudad_de_nacimiento'] == '', 'ciudad_de_nacimiento'] = np.nan

    print("Columna 'ciudad_de_nacimiento' limpiada.")
    print("Valores únicos en 'ciudad_de_nacimiento' (muestra de los más frecuentes):")
    print(df['ciudad_de_nacimiento'].value_counts().head(10)) # Muestra los 10 más frecuentes
else:
    print("Columna 'ciudad_de_nacimiento' no encontrada.")

Columna 'ciudad_de_nacimiento' limpiada.
Valores únicos en 'ciudad_de_nacimiento' (muestra de los más frecuentes):
ciudad_de_nacimiento
Mocoa           106
Quibdó           99
Palmira          99
Yopal            97
Ibagué           94
Barranquilla     93
Santa Marta      91
Florencia        91
Apartadó         90
Cúcuta           90
Name: count, dtype: int64


In [ ]:
if 'semestre' in df.columns:
    df['semestre'] = pd.to_numeric(df['semestre'], errors='coerce')
    # Validar que esté entre 1 y 10 (o el máximo del plan de estudios)
    df.loc[(df['semestre'] < 1) | (df['semestre'] > 10), 'semestre'] = np.nan # Se toma como referencias que 10 semestre es lo maximo.
    df['semestre'] = df['semestre'].astype('Int64')

    print("Columna 'semestre' limpiada.")
    print("Valores únicos en 'semestre':", df['semestre'].unique())
    print("Descripción de 'semestre':")
    print(df['semestre'].describe())
else:
    print("Columna 'semestre' no encontrada.")

Columna 'semestre' limpiada.
Valores únicos en 'semestre': <IntegerArray>
[1, 3, 2, 8, 7, 6, 5, 4, 9, <NA>]
Length: 10, dtype: Int64
Descripción de 'semestre':
count      3749.0
mean     4.728194
std      2.494927
min           1.0
25%           3.0
50%           5.0
75%           7.0
max           9.0
Name: semestre, dtype: Float64


In [ ]:
if 'promedio_actual' in df.columns:

    df['promedio_actual'] = df['promedio_actual'].astype(str)
    df['promedio_actual'] = df['promedio_actual'].str.replace(',', '.', regex=False)
    df['promedio_actual'] = pd.to_numeric(df['promedio_actual'], errors='coerce')
    df.loc[(df['promedio_actual'] < 0) | (df['promedio_actual'] > 5), 'promedio_actual'] = np.nan

    print("Columna 'promedio_actual' limpiada.")
    print("Descripción de 'promedio_actual':")
    print(df['promedio_actual'].describe())
else:
    print("Columna 'promedio_actual' no encontrada.")

Columna 'promedio_actual' limpiada.
Descripción de 'promedio_actual':
count    3792.000000
mean        3.652294
std         0.570611
min         2.800000
25%         3.200000
50%         3.600000
75%         4.000000
max         5.000000
Name: promedio_actual, dtype: float64


In [ ]:
if 'costo_matricula' in df.columns:

    df['costo_matricula'] = df['costo_matricula'].astype(str)
    df['costo_matricula'] = df['costo_matricula'].str.replace('.', '', regex=False) # Para separadores de miles
    df['costo_matricula'] = df['costo_matricula'].str.replace('$', '', regex=False) # Símbolo de peso
    df['costo_matricula'] = df['costo_matricula'].str.replace(r'[^\d]', '', regex=True)
    df['costo_matricula'] = pd.to_numeric(df['costo_matricula'], errors='coerce')

    df['costo_matricula'] = df['costo_matricula'].astype('Int64')

    print("Columna 'costo_matricula' limpiada.")
    print("Descripción de 'costo_matricula':")
    print(df['costo_matricula'].describe())
else:
    print("Columna 'costo_matricula' no encontrada.")

Columna 'costo_matricula' limpiada.
Descripción de 'costo_matricula':
count            3675.0
mean     2491183.673469
std       184253.079589
min           2100000.0
25%           2400000.0
50%           2500000.0
75%           2650000.0
max           2800000.0
Name: costo_matricula, dtype: Float64


Limpieza Específica por Columna

*   **edad:** Convertida a numérico Int64, valores fuera de rango (<=10 o >80) a NaN.
*   **genero:** Estandarizado a 'M', 'F', o NaN.
*   **ciudad_de_nacimiento:** Se corrigieron caracteres especiales (ej. 'MonterÃ­a' a 'Montería'), se eliminaron caracteres no alfabéticos y se capitalizó.
*   **semestre:** Convertido a numérico Int64, valores fuera de rango (1-10) a NaN.
*   **promedio_actual:** Convertido a numérico float, comas reemplazadas por puntos, valores fuera de rango (0-5) a NaN.
*   **costo_matricula:** Convertido a numérico Int64, eliminados puntos y símbolos de moneda.

In [ ]:
print("\nDataFrame final (primeras 5 filas):")
print(df.head())

print("\nInformación del DataFrame final:")
df.info()

print("\nResumen estadístico del DataFrame final:")
print(df.describe(include='all'))

print("\nConteo de valores NaN por columna en el DataFrame final:")
print(df.isnull().sum())


DataFrame final (primeras 5 filas):
  nombre_del_estudiante  edad fecha_de_nacimiento genero  \
1             abad vega    23          04/09/2002      F   
2       abaneth lizcano    22          30/11/1988      M   
3   abderraman martinez    18          02/11/2003      M   
4     abdruval collazos    22          21/08/1969      F   
5          abel sanchez    19          11/01/2006      M   

                                  email ciudad_de_nacimiento  \
1                  abadvega@miemail.com             Montería   
2      abanethlizcano@miuniversidad.com                Yumbo   
3  abderramanmartinez@miuniversidad.com             Rionegro   
4            abdruvalcollazos@mihot.com             Girardot   
5                 abelsanchez@mihot.com                Pasto   

  deporte_que_practica  semestre cursos_matriculados creditos_aprobados  \
1            Bádminton         1                   6                 65   
2           Baloncesto         3                 NaN               

In [ ]:
nombre_archivo_limpio = 'estudiantes_ingenieria_limpios.xlsx'
df_para_guardar = df.copy()
df_para_guardar.to_excel(nombre_archivo_limpio, index=False)
print(f"\nArchivo limpio guardado en el entorno de Colab como '{nombre_archivo_limpio}'")


Archivo limpio guardado en el entorno de Colab como 'estudiantes_ingenieria_limpios.xlsx'


In [ ]:
from google.colab import files
files.download(nombre_archivo_limpio)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**Paso 5: Generación del Archivo Limpio**

*   Se revisó el DataFrame final (primeras filas, información general, estadísticas y conteo de NaNs).
*   El DataFrame limpio se guardó en el archivo `estudiantes_ingenieria_limpios.xlsx`.
*   El archivo `estudiantes_ingenieria_limpios.xlsx` fue descargado a la computadora local.

